In [ ]:
!pip install -U duckduckgo_search
!pip install anthropic

In [ ]:

keywords = 'Wo steht der Dax?'
results = ddg(keywords, region='de', safesearch='On', time='m', max_results=3)

prompt=f"""Beantworten Sie die Frage am Ende des Textes anhand der folgenden Informationen:
### Eingabe
{{context}}
Frage: {{question}}
"""

formatted_prompt=prompt.format(context="\n".join([r["body"] for r in results]), question=keywords)
print(formatted_prompt)

In [ ]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents import load_tools

llm = OpenAI(temperature=0)
# search = GoogleSerperAPIWrapper()
tools = load_tools(["searx-search"], searx_host="http://localhost:8888", llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.run("What is the weather in Pomfret") the reigning men's U.S. Open champion?")

In [ ]:
print(agent.agent.llm_chain.prompt.template)

In [ ]:
agent.run("Wo steht der Dax?")

In [ ]:
from langchain.agents import tool
from duckduckgo_search import ddg

keywords = 'Wo steht der Dax?'

@tool("search", return_direct=True)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    results = ddg(query, region='de-de', safesearch='On', time='y', max_results=3)
    return "\n".join([r["body"] for r in results])

In [ ]:
from langchain.llms import Anthropic
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])


In [ ]:
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from duckduckgo_search import ddg

class CustomSearchTool(BaseTool):
    name = "Search"
    description = "useful to search the web you need to answer questions about current events"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return ddg(query, region='de-de', safesearch='On', time='y', max_results=3)
    
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Duckduck go does not support async")

In [ ]:
from langchain.llms import HuggingFaceHub, OpenAI,Anthropic
import os 

# llm = HuggingFaceHub(repo_id="google/flan-ul2", huggingfacehub_api_token=os.environ["HF_API_KEY"], model_kwargs={"do_sample": True, "max_new_tokens": 512, "top_p":0.8})
# llm = HuggingFaceHub(repo_id="OpenAssistant/oasst-sft-1-pythia-12b", huggingfacehub_api_token=os.environ["HF_API_KEY"], model_kwargs={"do_sample": True, "max_new_tokens": 512, "top_p":0.8})
# llm =OpenAI(temperature=0.7)

llm = Anthropic()
agent = initialize_agent([CustomSearchTool()], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [ ]:
agent.run("Who is the German chancellor?")